# Split into small Parquet files

In [15]:
sc.stop()

In [4]:
import sys
sys.path.append('./lib')
from numpy_pack import packArray,unpackArray
from binary_search import binary_search

from pyspark import SparkContext
sc = SparkContext(master=master_url,pyFiles=['lib/numpy_pack.py'])

In [10]:
# Open public bucket (readable by anybody with an AWS account, but not through a ublic URL)
bucket='mas-dse-public'
dir='/home/ec2-user/spark/'
s3helper.open_bucket(bucket)
from time import time

In [3]:
file_names=['US_Weather_Cleaned.parquet','Weather_Stations.parquet','US_Weather_cleaned_labeled.parquet']

In [9]:
t=time()
for file in file_names:
    !~/hadoop/bin/hadoop fs -rm -r /$file
    tmp = s3helper.s3_to_hdfs('/Weather/%s'%file, '/%s'%file)
    !~/hadoop/bin/hadoop fs -ls -h /$file
print 'copying took',time()-t,'seconds'
## copying took 62.695912838, 74.9509780407, 78.921860218 seconds

2017-03-13 02:51:47,576 INFO  [main] fs.TrashPolicyDefault (TrashPolicyDefault.java:initialize(92)) - Namenode trash configuration: Deletion interval = 0 minutes, Emptier interval = 0 minutes.
Deleted /US_Weather_Cleaned.parquet
Found 674 items
-rw-r--r--   3 ec2-user supergroup          0 2017-03-13 02:53 /US_Weather_Cleaned.parquet/US_Weather_Cleaned.parquet_$folder$
-rw-r--r--   3 ec2-user supergroup          0 2017-03-13 02:52 /US_Weather_Cleaned.parquet/_SUCCESS
-rw-r--r--   3 ec2-user supergroup    2909659 2017-03-13 02:52 /US_Weather_Cleaned.parquet/part-00000-387d8c85-08c5-4bd0-9794-3fd672e935c6.snappy.parquet
-rw-r--r--   3 ec2-user supergroup    2323441 2017-03-13 02:52 /US_Weather_Cleaned.parquet/part-00001-387d8c85-08c5-4bd0-9794-3fd672e935c6.snappy.parquet
-rw-r--r--   3 ec2-user supergroup    2713181 2017-03-13 02:52 /US_Weather_Cleaned.parquet/part-00002-387d8c85-08c5-4bd0-9794-3fd672e935c6.snappy.parquet
-rw-r--r--   3 ec2-user supergroup    2765879 2017-03-13 02:52 /US

In [17]:
# check what is in the S3 bucket
print s3helper.ls_s3()  # By default, list all files in the root directory of the bucket
print s3helper.ls_s3('Weather')

[u'Scripts', u'Scripts_$folder$', u'Spark-Data', u'Spark-Data_$folder$', u'Weather', u'Weather_$folder$', u'moby10b.txt']
[u'Weather/PCA_PRCP.pickle', u'Weather/PCA_SNOW.pickle', u'Weather/PCA_TMAX.pickle', u'Weather/STAT1.pickle', u'Weather/SampleStations.pickle', u'Weather/SampleStations_copy.pickle', u'Weather/US_Weather.parquet', u'Weather/US_Weather_Cleaned.parquet', u'Weather/US_Weather_Cleaned.parquet_$folder$', u'Weather/US_Weather_SSBBSBBS.csv.gz', u'Weather/US_Weather_cleaned_labeled.parquet', u'Weather/US_Weather_cleaned_labeled.parquet_$folder$', u'Weather/US_counts.pickle', u'Weather/Weather.parquet', u'Weather/Weather_Stations.parquet', u'Weather/small', u'Weather/test.json', u'Weather_$folder$']


In [12]:
# check what is on HDFS
!~/hadoop/bin/hadoop fs -du  /US_Weather_cleaned_labeled.parquet

17123295  /US_Weather_cleaned_labeled.parquet/.distcp.tmp.attempt_local1784583953_0001_m_000000_0
0         /US_Weather_cleaned_labeled.parquet/US_Weather_cleaned_labeled.parquet_$folder$
0         /US_Weather_cleaned_labeled.parquet/_SUCCESS
21284475  /US_Weather_cleaned_labeled.parquet/part-00000-534dbbed-3a0b-4a3f-9989-b18a821efc34.snappy.parquet
21047574  /US_Weather_cleaned_labeled.parquet/part-00001-534dbbed-3a0b-4a3f-9989-b18a821efc34.snappy.parquet
20232279  /US_Weather_cleaned_labeled.parquet/part-00002-534dbbed-3a0b-4a3f-9989-b18a821efc34.snappy.parquet
19544679  /US_Weather_cleaned_labeled.parquet/part-00003-534dbbed-3a0b-4a3f-9989-b18a821efc34.snappy.parquet
19264325  /US_Weather_cleaned_labeled.parquet/part-00004-534dbbed-3a0b-4a3f-9989-b18a821efc34.snappy.parquet
19131824  /US_Weather_cleaned_labeled.parquet/part-00005-534dbbed-3a0b-4a3f-9989-b18a821efc34.snappy.parquet
18988243  /US_Weather_cleaned_labeled.parquet/part-00006-534dbbed-3a0b-4a3f-9989-b18a821efc34.snappy.pa

In [5]:
# Start SQLContext and SparkSession
from pyspark import SparkContext
from pyspark.sql import SQLContext,Row,Column
from pyspark.sql import SparkSession
sqlContext = SQLContext(sc)
#spark=SparkSession(sc)

In [6]:
measurements=['TMAX','TMIN','TOBS','SNOW','SNWD','PRCP']

In [13]:
#read main data file
US_Weather_parquet='/US_Weather_cleaned_labeled.parquet/'
Query="SELECT * from parquet.`%s`"%US_Weather_parquet
print Query

SELECT * from parquet.`/US_Weather_cleaned_labeled.parquet/`


In [14]:
t=time()
df = sqlContext.sql(Query).cache()
print df.count()
print 'establishing SQL query took',time()-t,'seconds'

AnalysisException: u'Job aborted due to stage failure: Task 0 in stage 3.0 failed 4 times, most recent failure: Lost task 0.3 in stage 3.0 (TID 15, 172.31.23.3, executor 8): java.io.IOException: Could not read footer: java.lang.RuntimeException: hdfs://ec2-54-145-21-40.compute-1.amazonaws.com:9000/US_Weather_cleaned_labeled.parquet/US_Weather_cleaned_labeled.parquet_$folder$ is not a Parquet file (too small)\n\tat org.apache.parquet.hadoop.ParquetFileReader.readAllFootersInParallel(ParquetFileReader.java:248)\n\tat org.apache.spark.sql.execution.datasources.parquet.ParquetFileFormat$$anonfun$19.apply(ParquetFileFormat.scala:606)\n\tat org.apache.spark.sql.execution.datasources.parquet.ParquetFileFormat$$anonfun$19.apply(ParquetFileFormat.scala:595)\n\tat org.apache.spark.rdd.RDD$$anonfun$mapPartitions$1$$anonfun$apply$23.apply(RDD.scala:796)\n\tat org.apache.spark.rdd.RDD$$anonfun$mapPartitions$1$$anonfun$apply$23.apply(RDD.scala:796)\n\tat org.apache.spark.rdd.MapPartitionsRDD.compute

In [14]:
t=time()
print df.count()
print 'Running SQL query took',time()-t,'seconds'

SELECT * from parquet.`/US_Weather_cleaned_labeled.parquet`


AnalysisException: u'Job aborted due to stage failure: Task 0 in stage 1.0 failed 4 times, most recent failure: Lost task 0.3 in stage 1.0 (TID 7, 172.31.21.183, executor 2): java.io.IOException: Could not read footer: java.lang.RuntimeException: hdfs://ec2-54-198-205-1.compute-1.amazonaws.com:9000/US_Weather_cleaned_labeled.parquet/US_Weather_cleaned_labeled.parquet_$folder$ is not a Parquet file (too small)\n\tat org.apache.parquet.hadoop.ParquetFileReader.readAllFootersInParallel(ParquetFileReader.java:248)\n\tat org.apache.spark.sql.execution.datasources.parquet.ParquetFileFormat$$anonfun$19.apply(ParquetFileFormat.scala:606)\n\tat org.apache.spark.sql.execution.datasources.parquet.ParquetFileFormat$$anonfun$19.apply(ParquetFileFormat.scala:595)\n\tat org.apache.spark.rdd.RDD$$anonfun$mapPartitions$1$$anonfun$apply$23.apply(RDD.scala:796)\n\tat org.apache.spark.rdd.RDD$$anonfun$mapPartitions$1$$anonfun$apply$23.apply(RDD.scala:796)\n\tat org.apache.spark.rdd.MapPartitionsRDD.comput

In [22]:
from pyspark.sql.functions import *
Query='SELECT * from parquet.`/Weather_Stations.parquet`'
Station_TBL=sqlContext.sql(Query).cache()
Station_TBL.select(col('ID').alias('station'),'longitude','latitude').show(5)
#data.select(col("Name").alias("name"), col("askdaosdka").alias("age"))

+-----------+---------+--------+
|    station|longitude|latitude|
+-----------+---------+--------+
|US1COLR0185|-105.1144|  40.711|
|US1COLR0186|-105.0963| 40.8135|
|US1COLR0187| -105.064| 40.7689|
|US1COLR0189|-105.0242|  40.689|
|US1COLR0193|-105.0639| 40.6711|
+-----------+---------+--------+
only showing top 5 rows



In [ ]:
import numpy as np
row,=df.take(1)
unpackArray(row['vector'],data_type=np.float16)

In [ ]:
import numpy as np
t=time()

N=sc.defaultParallelism
print 'Number of executors=',N
#rdd0=df....cache().repartition(N)

# Number of executors= 80
# took 0.0209968090057 seconds

In [ ]:
t=time()
Stations=df.groupBy(df.station).count()
print 'took',time()-t,'seconds'

# took 1.15811896324 seconds
Stations.count()

In [ ]:
Stations.show(5)

In [ ]:
Stations2=Stations.join(Station_TBL.select(col('ID').alias('station'),'longitude','latitude','state'),on='station',how='left_outer')
Stations.count(), Stations2.count()

In [ ]:
Stations2.show(5)

In [ ]:
St_pandas=Stations2.toPandas()
tmp=St_pandas.sort_values('longitude')
print tmp.shape
print tmp.iloc[0,:]['longitude']
print tmp['longitude'][34738]
tmp.head()
#tmp.tail()


In [ ]:
def bisect(df,field):
    tmp=df.sort_values(field)
    l=tmp.shape[0]
    Min=tmp.iloc[0,:][field]
    Max=tmp.iloc[l-1,:][field]
    cumul=np.array(np.cumsum(tmp['count']))
    cumul=cumul/(cumul[-1]+0.0)
    midpoint=binary_search(cumul,0.5)
    #print cumul[midpoint-1:midpoint+1]
    Median=(tmp.iloc[midpoint-1,:][field]+tmp.iloc[midpoint,:][field])/2.
    return Min,Median,Max

In [ ]:
print bisect(St_pandas,'longitude')
print bisect(St_pandas,'latitude')

### Build a KD-tree.

In [ ]:
fields=['longitude','latitude']
field_i=0
def build_tree(DF,field_i,level,index):
    field=fields[field_i]
    Weight=DF['count'].sum()
    if level>0:
        min,thr,max=bisect(DF,field)
        sub1=DF[DF[field]>=thr]
        T1=build_tree(sub1,1-field_i,level-1,index+'B')
        sub2=DF[DF[field]<thr]
        T2=build_tree(sub2,1-field_i,level-1,index+'S')
        return {'min':min,
                'max':max,
                'level':level,
                'index':index,
                'field':field,
                'thr':thr,
                'Weight':Weight,
                'larger':T1,
                'smaller':T2
               }
    else:
        return Weight

In [ ]:
tree=build_tree(St_pandas,0,9,'')

In [ ]:
def calc_part(longitude, latitude,tree):
    """ Compute the part index for a given longitude and lattitude"""
    coords=[longitude,latitude]
    if tree['field']=='longitude':
        field_i=0
    else:
        field_i=1
    return find_recursive(tree,coords,field_i)

def find_recursive(tree,coords,field_i):
    if type(tree)!= dict:
        return None
    cond=coords[field_i]>tree['thr']
    if cond:
        R=find_recursive(tree['larger'],coords,1-field_i)
    else:
        R=find_recursive(tree['smaller'],coords,1-field_i)
    if R is None:
        return tree['index']
    else:
        return R

In [ ]:
from pyspark.sql.functions import udf
from pyspark.sql.types import StringType

label_udf = udf(lambda longit,lat:calc_part(longit,lat,tree),StringType())
df_withlabel=df.withColumn("label", label_udf(df.longitude,df.latitude))
df_withlabel.cache().count()

In [ ]:
label_counts=df_withlabel.groupBy(df_withlabel.label).count().collect()

In [ ]:
Labels=[r['label'] for r in label_counts]
len(Labels)

In [ ]:
%cd /home/ec2-user/Data
!ls -l
s3helper.local_to_s3('/home/ec2-user/Data/US_Weather_SSBBSBBS.csv.gz','/Weather/small/US_Weather_SSBBSBBS.csv.gz')
!rm *

In [ ]:

from pyspark.sql import SQLContext,Row,Column
from pyspark import SparkContext
from pyspark.sql import SparkSession

sqlContext = SQLContext(sc)
spark=SparkSession(sc)
print Query

In [ ]:
t=time()
df_withlabel = sqlContext.sql(Query).cache()
print df_withlabel.count()
print 'took',time()-t,'seconds'

In [ ]:
bucket='mas-dse-open'
s3helper.open_bucket(bucket)

In [ ]:
Labels_list=df_withlabel.select('label').collect()
Labels_list[:5]

In [ ]:
import pickle
for label in Labels:
    single_label=df_withlabel.filter(df_withlabel['label']==label).collect()
    print label,len(single_label),'  ',
    #print single_label[0]
    filename='/home/ec2-user/Data/US_Weather_%s.pickle'%label
    pickle.dump(single_label,open(filename,'wb'),protocol=2)
    !gzip $filename
    !ls -l $filename*
    filename='US_Weather_%s.pickle.gz'%label
    s3helper.local_to_s3('/home/ec2-user/Data/'+filename,'/Weather/small/'+filename)

In [ ]:
t=time()
#df_withlabel.write.save('hdfs:///US_Weather_cleaned_labeled.parquet')
#print 'HDFS:\n',s3helper.ls_hdfs()
files=s3helper.hdfs_to_s3("/US_Weather_cleaned_labeled.parquet", "/Weather/US_Weather_cleaned_labeled.parquet")
print 'S3:\n',s3helper.ls_s3('Weather')
print 'took',time()-t

In [ ]:
13000*4*365./1000000.

In [ ]:
# Compute the overall distribution of values and the distribution of the number of nan per year
def find_percentiles(SortedVals,percentile):
  L=len(SortedVals)/percentile
  return SortedVals[L],SortedVals[-L]
  
def computeOverAllDist(rdd0):
  UnDef=np.array(rdd0.map(lambda row:sum(np.isnan(row))).sample(False,0.01).collect())
  flat=rdd0.flatMap(lambda v:list(v)).filter(lambda x: not np.isnan(x)).cache()
  count,S1,S2=flat.map(lambda x: np.float64([1,x,x**2]))\
                  .reduce(lambda x,y: x+y)
  mean=S1/count
  std=np.sqrt(S2/count-mean**2)
  Vals=flat.sample(False,0.0001).collect()
  SortedVals=np.array(sorted(Vals))
  low100,high100=find_percentiles(SortedVals,100)
  low1000,high1000=find_percentiles(SortedVals,1000)
  return {'UnDef':UnDef,\
          'mean':mean,\
          'std':std,\
          'SortedVals':SortedVals,\
          'low100':low100,\
          'high100':high100,\
          'low1000':low100,\
          'high1000':high1000
          }

In [ ]:
def repack_array(row):
    arr=(np.array([np.float16(row[str(i)]) for i in range(1,366)]))
    #trim extremes
    trimmed=np.array([x if (x>low1000-1) and (x<high1000+1) else np.nan for x in arr],dtype=np.float16)
    #count nan
    count=int(np.sum(np.isnan(trimmed)))
    #return row with 365 entries replaced with a bytestream representing a bytearray.

    New={}
    keep_keys=['longitude', 'station', 'latitude', 'year', 'elevation', 'measurement']
    for k in keep_keys:
        New[k]=row[k]
    New['vector']=packArray(trimmed)
    New['undefs']=count
    return Row(**New)


In [ ]:
from numpy import linalg as LA

STAT={}  # dictionary storing the statistics for each measurement
df={}
clean_df={}

for meas in measurements:
    t=time()
    Query="SELECT * FROM parquet.`%s`\n\tWHERE measurement = '%s'"%(US_Weather_parquet,meas)
    print Query
    df[meas] = sqlContext.sql(Query)
    print 'size of dataframe for ',meas,'before filtering =',df[meas].count()
    rdd0=df[meas].rdd.map(lambda row:(row['station'],((row['measurement'],row['year']),np.array([np.float64(row[str(i)]) for i in range(1,366)])))).cache()

    rdd1=rdd0.sample(False,1)\
           .map(lambda (key,val): val[1])\
           .cache()\
           .repartition(N)

    STAT[meas]=computeOverAllDist(rdd1)   # Compute the statistics 
    low1000 = STAT[meas]['low1000']  # unpack the extreme values statistics
    high1000 = STAT[meas]['high1000']

    Cleaned=df[meas].rdd.map(lambda row:repack_array(row))
    
    clean_df[meas]=spark.createDataFrame(Cleaned).filter("undefs<50")

    print 'size of dataframe for ',meas,'after filtering =',clean_df[meas].count()
    print meas,'took',time()-t,'seconds'



In [ ]:
import sys
sys.path.append('./lib')
from numpy_pack import packArray,unpackArray



In [ ]:
t=time()
df_all = clean_df['TMIN']
s=clean_df['TMIN'].count()
print 'TMIN',s
for key in ['TOBS', 'TMAX', 'SNOW', 'SNWD', 'PRCP']:
    df_all=df_all.union(clean_df[key])
    s1= clean_df[key].count()
    s+= s1
    print key,s1,s
print 'total size=',df_all.cache().count()
print 'took',time()-t,'seconds'

In [ ]:
t=time()
#df_all.write.save("hdfs:///US_Weather_Cleaned.parquet")
print 'HDFS:\n',s3helper.ls_hdfs()
s3helper.hdfs_to_s3("/US_Weather_Cleaned.parquet", "/Weather/US_Weather_Cleaned.parquet")
print 'S3:\n',s3helper.ls_s3('Weather')
print 'took',time()-t
# took took 525.435763121